# Processing & Pruning Our Data

## Import our dependencies

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import os
from sklearn.impute import SimpleImputer
import json

## Load our datasets

In [2]:
atlas_fifteen_path = '../data/raw/Raw_Atlas_2015/Raw_Atlas_Data_2015.xlsx'
fifteen_xl = pd.ExcelFile(atlas_fifteen_path)
atlas_ten_path = '../data/raw/Raw_Atlas_2010/Raw_Atlas_Data_2010.xlsx'
ten_xl = pd.ExcelFile(atlas_ten_path)

atlas_nineteen = pd.read_csv('../data/raw/Raw_Atlas_2019/Raw_Atlas_Data_2019.csv')
atlas_fifteen = fifteen_xl.parse('Food Access Research Atlas')
atlas_ten = ten_xl.parse('Food Access Research Atlas')

## Processing 2019 data for our present-day classification model

The 100+ columns in our table are often redundant and useless for training our model.

Let's keep the fundamentals, along with some general data buffer zones for identifying relative isolation + outliers.

In [3]:
keep_cols = [
    'Urban', 'PovertyRate', 'MedianFamilyIncome',
    'TractLOWI', 'TractKids', 'TractSeniors', 'TractHUNV', 'TractSNAP', 
]

buffer_cols = [
    'lapop1share', 'lalowi1share', 'lakids1share', 'laseniors1share',
    'lahunv1share', 'lasnap1share'
]

### Calculating & fixing missing data values within our set

In [4]:
processed_atlas_nineteen = atlas_nineteen[keep_cols + buffer_cols + ['Pop2010']].copy()
nan_count = processed_atlas_nineteen.isna().sum()
print(nan_count)

Urban                     0
PovertyRate               3
MedianFamilyIncome      748
TractLOWI                 4
TractKids                 4
TractSeniors              4
TractHUNV                 4
TractSNAP                 4
lapop1share           19989
lalowi1share          19989
lakids1share          19989
laseniors1share       19989
lahunv1share          19966
lasnap1share          19966
Pop2010                   0
dtype: int64


In [5]:
imputer = SimpleImputer(strategy='median')
processed_atlas_nineteen[keep_cols] = imputer.fit_transform(processed_atlas_nineteen[keep_cols])

In [6]:
rural_missing = processed_atlas_nineteen[processed_atlas_nineteen['Urban'] == 0][buffer_cols].isna().mean()
urban_missing = processed_atlas_nineteen[processed_atlas_nineteen['Urban'] == 1][buffer_cols].isna().mean()

print("Rural missing fraction:\n", rural_missing)
print("Urban missing fraction:\n", urban_missing)

Rural missing fraction:
 lapop1share        0.000346
lalowi1share       0.000346
lakids1share       0.000346
laseniors1share    0.000346
lahunv1share       0.000346
lasnap1share       0.000346
dtype: float64
Urban missing fraction:
 lapop1share        0.362214
lalowi1share       0.362214
lakids1share       0.362214
laseniors1share    0.362214
lahunv1share       0.361797
lasnap1share       0.361797
dtype: float64


When observing the data, we can see that our buffer zones typically return NaN when in an urban environment.

Realistically, there seems to be no reason to NOT impute our missing urban values.
Urban environments realistically do not come across notable volatilities that would return an entirely new result if not tracked properly in our database.
As a result, median imputation will be used for these values as well, especially considering that we can use the respective ~ 60% (~ 48k) reference values.

In [7]:
processed_atlas_nineteen[buffer_cols] = imputer.fit_transform(processed_atlas_nineteen[buffer_cols])
nan_count = processed_atlas_nineteen.isna().sum()
print(nan_count)

Urban                 0
PovertyRate           0
MedianFamilyIncome    0
TractLOWI             0
TractKids             0
TractSeniors          0
TractHUNV             0
TractSNAP             0
lapop1share           0
lalowi1share          0
lakids1share          0
laseniors1share       0
lahunv1share          0
lasnap1share          0
Pop2010               0
dtype: int64


### Additional features for our 2019 model

#### Simple ratios:

In [8]:
processed_atlas_nineteen['LOWIRatio'] = processed_atlas_nineteen['TractLOWI'] / processed_atlas_nineteen['Pop2010'] # Low income percentage
processed_atlas_nineteen['SNAPRatio'] = processed_atlas_nineteen['TractSNAP'] / processed_atlas_nineteen['Pop2010'] # Percentage of residents receiving SNAP
processed_atlas_nineteen['HUNVRatio'] = processed_atlas_nineteen['TractHUNV'] / processed_atlas_nineteen['Pop2010'] # Percentage of residents without a vehicle (important for food deserts!)
processed_atlas_nineteen['FoodInsecurityIndex'] = (
    processed_atlas_nineteen['LOWIRatio'] +
    processed_atlas_nineteen['SNAPRatio'] +
    processed_atlas_nineteen['HUNVRatio']
)

#### More complex features:

In [9]:
processed_atlas_nineteen['SNAPDisparity'] = processed_atlas_nineteen['SNAPRatio'] - processed_atlas_nineteen['lasnap1share']

Our calculated SNAP disparity is how much higher/lower the tract's SNAP is relative to its surrounding area.

A high positive disparity would indicate isolated food insecurity, while a negative disparity would show a priveleged pocket inside of a worse-off area.

In [10]:
processed_atlas_nineteen['LOWIWeighted'] = processed_atlas_nineteen['TractLOWI'] * processed_atlas_nineteen['PovertyRate']

Our weighted low income emphasizes tracts with both a high number of low-income people and a high poverty rate.

This helps create more granular priorization for our model. 

### Scaling our 2019 data before saving to be processed later

In [11]:
scaler = MinMaxScaler()
scaled_processed_atlas_nineteen = scaler.fit_transform(processed_atlas_nineteen)

I chose to use sklearn's MinMaxScaler() module over the StandardScaler() module for this project.

Given our present-day forecasting model will be using a ReLU for the hidden layer activation function, MinMax scaling shines in this aspect.

Our K-Means clustering would work fine with either choice. Given that the feature values in this dataset (proportions, demographic shares, poverty rates) are naturally bounded and positive, MinMaxScaler could also be seen as a natural fit.

### Save our processed data to be ran through our autoencoder/K-means model

In [12]:
final_atlas_nineteen = pd.DataFrame(scaled_processed_atlas_nineteen, columns=processed_atlas_nineteen.columns)
final_atlas_nineteen.head()

,Urban,PovertyRate,MedianFamilyIncome,TractLOWI,TractKids,TractSeniors,TractHUNV,TractSNAP,lapop1share,lalowi1share,...,laseniors1share,lahunv1share,lasnap1share,Pop2010,LOWIRatio,SNAPRatio,HUNVRatio,FoodInsecurityIndex,SNAPDisparity,LOWIWeighted
0,1.0,0.113,0.318183,0.036220,0.042803,0.012796,0.000990,0.046897,0.9919,0.2411,...,0.1144,0.0079,0.1322,0.051027,0.007051,0.022840,0.000292,0.008725,0.849223,0.006762
1,1.0,0.179,0.187881,0.063843,0.051161,0.012391,0.014689,0.071724,0.5811,0.2783,...,0.0583,0.0900,0.1295,0.057916,0.010951,0.030778,0.003815,0.014296,0.852045,0.018882
2,1.0,0.150,0.242867,0.103964,0.075475,0.025418,0.016339,0.079080,0.4600,0.1418,...,0.0596,0.0000,0.0587,0.090038,0.011472,0.021832,0.002730,0.013853,0.921084,0.025766
3,1.0,0.028,0.275182,0.073396,0.085690,0.052342,0.003466,0.045057,0.3109,0.0783,...,0.0539,0.0046,0.0176,0.117086,0.006229,0.009566,0.000445,0.007032,0.961000,0.003395
4,1.0,0.152,0.379128,0.178475,0.266948,0.065196,0.037960,0.155862,0.2455,0.0545,...,0.0336,0.0135,0.0204,0.287442,0.006170,0.013481,0.001987,0.007736,0.958351,0.044822


In [13]:
final_atlas_nineteen.to_csv("../data/processed/processed_atlas_nineteen.csv")

## Processing our 2015 & 2010 data

### Running through our 2015 data (same steps as 2019)

In [14]:
processed_atlas_fifteen = atlas_fifteen[keep_cols + buffer_cols + ['POP2010']].copy()
# The 2015 Atlas uses the naming convention 'POP2010' opposed to the 2019 variation of 'Pop2010'

In [15]:
processed_atlas_fifteen[keep_cols + buffer_cols] = imputer.fit_transform(processed_atlas_fifteen[keep_cols + buffer_cols])

In [16]:
processed_atlas_fifteen['LOWIRatio'] = processed_atlas_fifteen['TractLOWI'] / processed_atlas_fifteen['POP2010'] # Low income percentage
processed_atlas_fifteen['SNAPRatio'] = processed_atlas_fifteen['TractSNAP'] / processed_atlas_fifteen['POP2010'] # Percentage of residents receiving SNAP
processed_atlas_fifteen['HUNVRatio'] = processed_atlas_fifteen['TractHUNV'] / processed_atlas_fifteen['POP2010'] # Percentage of residents without a vehicle (important for food deserts!)
processed_atlas_fifteen['FoodInsecurityIndex'] = (
    processed_atlas_fifteen['LOWIRatio'] +
    processed_atlas_fifteen['SNAPRatio'] +
    processed_atlas_fifteen['HUNVRatio']
)
processed_atlas_fifteen['SNAPDisparity'] = processed_atlas_fifteen['SNAPRatio'] - processed_atlas_fifteen['lasnap1share']
processed_atlas_fifteen['LOWIWeighted'] = processed_atlas_fifteen['TractLOWI'] * processed_atlas_fifteen['PovertyRate']

### Running through our 2010 data

In [17]:
pd.set_option('display.max_columns', None)
atlas_ten.head()

,CensusTract,State,County,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,Urban,Rural,LA1and10,LAhalfand10,LA1and20,LATracts_half,LATracts1,LATracts10,LATracts20,LATractsVehicle_20,HUNVFlag,GroupQuartersFlag,OHU2010,NUMGQTRS,PCTGQTRS,LowIncomeTracts,POP2010,UATYP10,lapophalf,lapophalfshare,lalowihalf,lalowihalfshare,lakidshalf,lakidshalfshare,laseniorshalf,laseniorshalfshare,lahunvhalf,lahunvhalfshare,lapop1,lapop1share,lalowi1,lalowi1share,lakids1,lakids1share,laseniors1,laseniors1share,lahunv1,lahunv1share,lapop10,lapop10share,lalowi10,lalowi10share,lakids10,lakids10share,laseniors10,laseniors10share,lahunv10,lahunv10share,lapop20,lapop20share,lalowi20,lalowi20share,lakids20,lakids20share,laseniors20,laseniors20share,lahunv20,lahunv20share
0,1001020100,AL,Autauga,0,0,0,0,1,0,1,1,1,1,1,0,0,0,0,0,693,0.0,0.000000,0,1912,U,1732.225468,0.905976,306.546737,0.160328,466.426429,0.919973,198.828220,0.899675,44.212100,0.063798,1357.480940,0.709980,245.277225,0.128283,363.638381,0.717235,162.497246,0.735282,31.579173,0.045569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001020200,AL,Autauga,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,743,181.0,0.083410,0,2170,U,1410.374828,0.649942,484.905037,0.223459,448.163512,0.739544,139.305390,0.650960,86.423433,0.116317,483.429683,0.222779,170.838823,0.078728,174.770469,0.288400,50.976822,0.238209,34.398590,0.046297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001020300,AL,Autauga,0,0,0,0,1,0,1,1,1,1,1,0,0,0,0,0,1256,0.0,0.000000,0,3373,U,2764.604126,0.819628,773.419284,0.229297,744.891575,0.833212,346.203097,0.788618,54.188593,0.043144,1417.874893,0.420360,380.786290,0.112892,377.128132,0.421844,190.001480,0.432805,19.156371,0.015252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1001020400,AL,Autauga,0,0,0,0,1,0,1,1,1,1,1,0,0,0,0,0,1722,0.0,0.000000,0,4386,U,4272.112205,0.974034,874.067405,0.199286,980.143479,0.965659,892.805993,0.987617,16.964191,0.009851,1909.275364,0.435311,311.160977,0.070944,470.411544,0.463460,374.051202,0.413773,3.926144,0.002280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1001020500,AL,Autauga,0,0,0,0,1,0,1,1,1,1,1,0,0,1,1,0,4082,181.0,0.016812,0,10766,U,7798.993990,0.724410,1131.052984,0.105058,2314.376847,0.731934,843.200608,0.748846,177.089308,0.043383,2753.648392,0.255773,373.426978,0.034686,745.740558,0.235845,373.826962,0.331996,57.430973,0.014069,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


This sucks! 
It seems like our 2010 atlas is missing SNAP, Seniors, Kids, and Income breakdowns at the tract level. 

All of these are present in both our 2015 & 2019 models.
    
To recap our model necessities, our random forest forecasing model need to predict the 'Food Desert' status of a tract given fed labels supplied by our autoencoder/k-means clustering model.

For the purposes of this project, we will need to find a reasonable workaround to somehow impute these features while not skewing data at the same time.
Sadly, I don't think some basic imputing will do the trick here.

In [18]:
keep_cols_2010 = [
    'Urban',
    'LowIncomeTracts',
    'HUNVFlag',
    'POP2010'
]
buffer_cols_2010 = [
    'lapop1share',
    'lalowi1share',
    'lakids1share',
    'laseniors1share',
    'lahunv1share'
]

These are basic features which are shared between all of our datasets.
Asides from these, the similarities end.

In [19]:
processed_atlas_ten = atlas_ten[keep_cols_2010 + buffer_cols_2010].copy()

#### Feature engineering our missing columns

##### PovertyRate

In [20]:
processed_atlas_ten['PovertyRate_Proxy'] = (
    0.5 * atlas_ten['lalowi1share'] +
    0.3 * atlas_ten['lalowi10share'] +
    0.2 * atlas_ten['lalowi20share']
) * 100

The methodology used in estimating PovertyRate_Proxy is that by weighting percentages of populations close to a supermarket based on distance, we are able to mimic a broader poverty signal.

##### MedianIncome

In [21]:
processed_atlas_ten['MedianIncome_Proxy'] = (1 - atlas_ten['lalowi1share']) * 10000

The logic used in estimating MedianIncome_Proxy is that since higher lalowi1share implies lower median income, we can can use an inverse relationship to build a rough proxy.

##### TractLOWI_Proxy

In [22]:
processed_atlas_ten['TRACTLOWI_Proxy'] = (
    atlas_ten['lalowi1'] +
    atlas_ten['lalowi10'] +
    atlas_ten['lalowi20']
) / 3

The logic used in estimating TractLOWI_Proxy is that by averaging the general amount of low income people across various ranges of buffers, we can create a good idea of what this statistic would come out to.

##### TRACTKids_Proxy, TRACTSeniors_Proxy, and TRACTHunv_Proxy

In [23]:
processed_atlas_ten['TRACTKids_Proxy'] = (
    atlas_ten['lakids1'] +
    atlas_ten['lakids10'] +
    atlas_ten['lakids20']
) / 3
processed_atlas_ten['TractSeniors_Proxy'] = (
    atlas_ten['laseniors1'] +
    atlas_ten['laseniors10'] +
    atlas_ten['laseniors20']
) / 3
processed_atlas_ten['TractHUNV_Proxy'] = (
    atlas_ten['lahunv1'] +
    atlas_ten['lahunv10'] +
    atlas_ten['lahunv20']
) / 3

Given that TRACTKids, TRACTSeniors, and TRACTHUNV are statistics characterized by an actual count, rather than a ratio, we are able to approximate them using the same logic as TractLOWI_Proxy.

##### TRACTSNAP

Sadly, our 2010 dataset is completely devoid of any SNAP data, and as a result, feature engineering/estimating SNAP statistics is virtually impossible.

To solve this, feature engineering will be done with the help of the 2010 ACS (American Community Survey), which contains needed SNAP data. 

In [24]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv("CENSUS_API_KEY")

Loading in our API key from the environment folder.

##### Fetching API Census data for TRACTSNAP

In [30]:
import requests
import pandas as pd
from typing import Optional, List

def fetch_SNAP_state(state_index: str, API_KEY: str) -> Optional[pd.Series]:
    BASE_URL = "https://api.census.gov/data/2010/acs/acs5"
    FIELDS = ["B22003_001E", "B22003_002E"]  # B22003_001E: Total households, B22003_002E: Households with SNAP
    GEOGRAPHY = "tract:*"
    
    params = {
        "get": ",".join(FIELDS),
        "for": GEOGRAPHY,
        "in": f"state:{state_index}",
        "key": API_KEY
    }

    response = requests.get(BASE_URL, params=params)

    if response.status_code != 200:
        print(f"API request failed with status code {response.status_code} for state {state_index}.")
        return None

    try:
        data = response.json()
    except requests.exceptions.JSONDecodeError as e:
        print(f"JSON parsing failed for state {state_index}: {e}")
        return None

    if not data or len(data) < 2:
        print(f"No data returned or unexpected response format for state {state_index}.")
        return None

    columns = data[0]
    values = data[1:]
    tract_snap = pd.DataFrame(values, columns=columns)
    required_geo_cols = ['state', 'county', 'tract']
    if all(col in tract_snap.columns for col in required_geo_cols):
        tract_snap["CensusTract"] = tract_snap["state"] + tract_snap["county"] + tract_snap["tract"]
        tract_snap = tract_snap.set_index("CensusTract")
    else:
        print(f"Warning: Missing one or more of {required_geo_cols} columns in response for state {state_index}.")
        return None 
    total_households = pd.to_numeric(tract_snap[FIELDS[0]], errors='coerce')
    snap_households = pd.to_numeric(tract_snap[FIELDS[1]], errors='coerce')

    snap_rate = pd.Series(0.0, index=tract_snap.index, dtype=float)

    valid_mask = (total_households > 0) & (total_households.notna())
    
    snap_rate.loc[valid_mask] = snap_households.loc[valid_mask] / total_households.loc[valid_mask]
    return snap_rate


In [31]:
state_abbrev_to_fips = {
    'AL': '01', 'AK': '02', 'AZ': '04', 'AR': '05', 'CA': '06',
    'CO': '08', 'CT': '09', 'DE': '10', 'DC': '11', 'FL': '12',
    'GA': '13', 'HI': '15', 'ID': '16', 'IL': '17', 'IN': '18',
    'IA': '19', 'KS': '20', 'KY': '21', 'LA': '22', 'ME': '23',
    'MD': '24', 'MA': '25', 'MI': '26', 'MN': '27', 'MS': '28',
    'MO': '29', 'MT': '30', 'NE': '31', 'NV': '32', 'NH': '33',
    'NJ': '34', 'NM': '35', 'NY': '36', 'NC': '37', 'ND': '38',
    'OH': '39', 'OK': '40', 'OR': '41', 'PA': '42', 'RI': '44',
    'SC': '45', 'SD': '46', 'TN': '47', 'TX': '48', 'UT': '49',
    'VT': '50', 'VA': '51', 'WA': '53', 'WV': '54', 'WI': '55',
    'WY': '56'
}

In [32]:
snap_dfs = []

for abbrev in atlas_ten["State"].unique():
    state_fips = state_abbrev_to_fips.get(abbrev)
    if not state_fips:
        print(f"Unknown state abbreviation: {abbrev}")
        continue

    print(f"Fetching for state: {abbrev} (FIPS: {state_fips})")

    try:
        snap_df = fetch_SNAP_state(state_fips, API_KEY)
        snap_dfs.append(snap_df)
    except Exception as e:
        print(f"Failed for state {abbrev}: {e}")

Fetching for state: AL (FIPS: 01)
Fetching for state: AK (FIPS: 02)
Fetching for state: AZ (FIPS: 04)
Fetching for state: AR (FIPS: 05)
Fetching for state: CA (FIPS: 06)
Fetching for state: CO (FIPS: 08)
Fetching for state: CT (FIPS: 09)
Fetching for state: DE (FIPS: 10)
Fetching for state: DC (FIPS: 11)
Fetching for state: FL (FIPS: 12)
Fetching for state: GA (FIPS: 13)
Fetching for state: HI (FIPS: 15)
Fetching for state: ID (FIPS: 16)
Fetching for state: IL (FIPS: 17)
Fetching for state: IN (FIPS: 18)
Fetching for state: IA (FIPS: 19)
Fetching for state: KS (FIPS: 20)
Fetching for state: KY (FIPS: 21)
Fetching for state: LA (FIPS: 22)
Fetching for state: ME (FIPS: 23)
Fetching for state: MD (FIPS: 24)
Fetching for state: MA (FIPS: 25)
Fetching for state: MI (FIPS: 26)
Fetching for state: MN (FIPS: 27)
Fetching for state: MS (FIPS: 28)
Fetching for state: MO (FIPS: 29)
Fetching for state: MT (FIPS: 30)
Fetching for state: NE (FIPS: 31)
Fetching for state: NV (FIPS: 32)
Fetching for s

In [38]:
all_snap_data = pd.concat(snap_dfs, ignore_index=True)
all_snap_data.columns = ['index', 'TRACTSNAP']
all_snap_data.head()

0    0.057471
1    0.089041
2    0.086247
3    0.063078
4    0.037423
dtype: float64

#### What does the above code do?
This code pulls 2010 SNAP (food stamp) data from the U.S. Census API for each state in atlas_ten. 

To summarize, it loads the API key (which you'll need to provide in a .env file), defines a function to fetch SNAP and total household counts per census tract, calculates SNAP participation rates, and loops through all states using FIPS codes to retrieve and store the results in the snap_dfs dataframe.